In [1]:
from datasets import load_dataset

dataset = load_dataset("harsh4248/human_vs_llm")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
def extract_labels(dataset_split):
    return [example['label'] for example in dataset_split]

train_labels = extract_labels(dataset['train'])
all_labels = train_labels

In [3]:
set(all_labels)

{'Bloom-7B',
 'Claude-Instant-v1',
 'Claude-v1',
 'Cohere-Command',
 'Dolphin-2.5-Mixtral-8x7B',
 'Dolphin-Mixtral-8x7B',
 'Falcon-180B',
 'Flan-T5-Base',
 'Flan-T5-Large',
 'Flan-T5-Small',
 'Flan-T5-XL',
 'Flan-T5-XXL',
 'GLM-130B',
 'GPT-3.5',
 'GPT-4',
 'GPT-J',
 'GPT-NeoX',
 'Gemini-Pro',
 'Goliath-120B',
 'Human',
 'LLaMA-13B',
 'LLaMA-2-70B',
 'LLaMA-2-7B',
 'LLaMA-30B',
 'LLaMA-65B',
 'LLaMA-7B',
 'LZLV-70B',
 'Mistral-7B',
 'Mistral-7B-OpenOrca',
 'Mixtral-8x7B',
 'MythoMax-L2-13B',
 'Neural-Chat-7B',
 'Noromaid-20B',
 'Nous-Capybara-34B',
 'Nous-Capybara-7B',
 'Nous-Hermes-LLaMA-2-13B',
 'Nous-Hermes-LLaMA-2-70B',
 'OPT-1.3B',
 'OPT-125M',
 'OPT-13B',
 'OPT-2.7B',
 'OPT-30B',
 'OPT-350M',
 'OPT-6.7B',
 'OpenChat-3.5',
 'OpenHermes-2-Mistral-7B',
 'OpenHermes-2.5-Mistral-7B',
 'PaLM-2',
 'Psyfighter-13B',
 'Psyfighter-2-13B',
 'RWKV-5-World-3B',
 'StripedHyena-Nous-7B',
 'T0-11B',
 'T0-3B',
 'Text-Ada-001',
 'Text-Babbage-001',
 'Text-Curie-001',
 'Text-Davinci-001',
 'Text-Da

changing labels to 0 if written by human and changing to 1 if written by ai

In [4]:
def modify_labels(example):
    example['label'] = 0 if example['label'] == 'Human' else 1
    return example

# Apply the label modification function to the dataset
dataset = dataset.map(modify_labels)

In [5]:
from datasets import load_dataset, DatasetDict
train_testvalid = dataset['train'].train_test_split(test_size=0.2, seed=42)
train_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': train_testvalid['test']
})

In [6]:
train_valid_dataset['test'][1]

{'label': 0,
 'title': "First off, let me make something clear: I'm not an anti-vaccer, and I dislike anti-vaccers. However, children do not have control over whether or not they're vaccinated; parents do. If a child is kept out of school during an outbreak, it is absolutely not their fault, yet one would end up being punished for it: first, academic performance will likely suffer. Second, one may be seen as an outcast whose parents are kind of nutty, and that may influence one's social life. Additionally, it is unjust that the parents, whose taxes pay for public schools (and who may not be able to afford other schools), will not be given the option to have their child go a public school. I believe that this is different from, for example, someone who pays taxes, but whose children go to a private school, because in that case, the parents still have the option of sending their children to a public school. However, I agree that sending unvaccinated children to school will increase the r

In [7]:
train_valid_dataset['train'][0]

{'label': 1,
 'title': "In the intricate tapestry of modern genetic science, the threads of ethical considerations are woven tightly with those of technological advancements. The evolution of forensic genetic genealogy is no exception, as it stands at the nexus of law enforcement, personal privacy, and historical rectification. This scientific technique, which uses DNA to trace lineage and ancestry, has far-reaching implications, especially in the realms of family reunification and reparations. The ethical landscape of these applications is a complex one, fraught with myriad questions about the rights of individuals, families, and communities.\n\nForensic genetic genealogy has gained public attention largely through its role in solving cold cases, where law enforcement agencies use public DNA databases to find relatives of suspects and ultimately identify the culprits. The success stories are compelling, bringing long-sought closure to victims' families. However, the use of genetic gen

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True)

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['title'], padding="max_length", truncation=True)

tokenized_datasets = train_valid_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create the Trainer
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],


    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Save the model
model.save_pretrained("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-model")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]